In [1]:
#Definindo finais de semana
mes = "09"
lista_fds = ['01','07','08','14','15','21','22','28','29']

# Biblioteca e Funções

In [2]:
####################################-BIBLIOTECAS-####################################

# Manipulação e Tratamento de dados
import openpyxl
import pandas as pd
import numpy as np
from numpy import NaN

#ignorando Warning inuteis
import warnings 
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

#string e data
import unidecode #limpeza
import re #regex

#tempo
import time
from time import sleep

#web scraping
import requests
from bs4 import BeautifulSoup

#automação
from selenium import webdriver #navegador
from selenium.webdriver.common.keys import Keys #comandos do teclado
from selenium.webdriver.common.by import By 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait #esperar
from selenium.webdriver.support.expected_conditions import (visibility_of, 
                                                           staleness_of, 
                                                           invisibility_of_element)
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

import pyautogui #

#manipulação do diretorio
import os

In [3]:
# FUNÇÕES XPATH
def xpath_element(xpath):
    return browser.find_element("xpath", xpath)

def xpath_click(xpath):
    try:
        xpath_element(xpath).click()
    except:
        sleep(2)
        xpath_element(xpath).click()
    
def xpath_send_keys(xpath, texto):
    xpath_element(xpath).send_keys(texto)
    
def xpath_clear(xpath):
    xpath_element(xpath).clear()

def existencia(caminho):
    elementos = browser.find_elements(By.XPATH, caminho)
    return bool(elementos)

In [4]:
#FUNÇÕES
def count_freq(data_frame, coluna):
    freq = data_frame[coluna].value_counts()
    perc = data_frame[coluna].value_counts(normalize = True)*100
    tabela = pd.DataFrame({"Freq":freq, "Perc":perc})
    return display(tabela)

def salvando_nome_arquivo(nome_arquivo, dataframe):
    if os.path.exists(nome_arquivo): #O nome_arquivo existe
        df1 = pd.read_excel(nome_arquivo)
        len_antigo = len(dataframe)
        
        dataframe = pd.concat([df1, dataframe]).reset_index(drop=True)
        dataframe = dataframe.drop_duplicates(subset=['chat'])
        
        print(nome_arquivo, '# arquivo atualizado #, len =', len(df1), " += ", len_antigo, "=>", len(dataframe))
        
        dataframe.to_excel(nome_arquivo, index=False)
    
    else: #O nome_arquivo não existe
        dataframe.to_excel(nome_arquivo, index=False)
        print(nome_arquivo, '# arquivo criado #, len =', len(dataframe)) 

# Ligando navegador

In [5]:
#browser.quit(); sleep(5)
browser = webdriver.Firefox()
action_chains = ActionChains(browser)

#fazendo login
browser.get('https://www.pegaplantao.com.br/login')
sleep(1)
xpath_send_keys('//*[@id="MainContent_LoginUser_UserName"]', 'gabriel_s_anjos@yahoo.com') #e-mail
xpath_send_keys('//*[@id="MainContent_LoginUser_Password"]', 'Gabriel2022!') #senha
xpath_click('//*[@id="MainContent_LoginUser_btnLogin"]') #login

In [6]:
browser.get('https://www.pegaplantao.com.br/EscalaMensal/old')

In [7]:
#click setor/profissional
xpath_click('//*[@id="s2id_autogen1"]/a/span[1]')

#click profissional
xpath_click('//*[@id="select2-drop"]/ul/li[2]/div')

# Capturando setores:

In [8]:
#click criar novo plantão
def criar_plantao():
    xpath_click('//*[@id="shift-monthly"]/div[2]/table/tbody/tr[1]/td[4]/button')
criar_plantao()

def esperar_elemento_aparecer(elemento, tempo):
    sleep(0.5)
    WebDriverWait(browser, tempo).until(visibility_of(xpath_element(elemento)))

sleep(2)
esperar_elemento_aparecer('//*[@id="service-ckb-list-all-owner"]', 10)
print('apareceu')

def esperar_elemento_desaparecer(elemento, tempo):
    sleep(0.5)
    WebDriverWait(browser, tempo).until_not(visibility_of(xpath_element(elemento)))

apareceu


In [9]:
#CAPTURANDO DADOS DO UNIDADE E SETOR
xpath_click('//*[@id="s2id_ddlServiceSector"]/a/span[1]')
setor_html = xpath_element('//*[@id="select2-drop"]/ul').get_attribute('outerHTML')
soup = BeautifulSoup(setor_html, 'html.parser')

lista_unidade = []
lista_unidade_count = []
lista_plt = []
lista_plt_count = []

count_unidade = 1

for i in soup.find_all(class_="select2-results-dept-0 select2-result select2-result-unselectable select2-result-with-children"):
    #print('\n', i.find(class_='select2-result-label').get_text(), '====== \n')
    count_plt = 1
    for t in i.find_all(class_='select2-results-dept-1 select2-result select2-result-selectable'): 
        #print(t.get_text())
        lista_plt.append(t.get_text())
        lista_plt_count.append(count_plt)
        count_plt += 1

        lista_unidade.append(i.find(class_='select2-result-label').get_text())
        lista_unidade_count.append(count_unidade)
    count_unidade += 1

scrap_unidades = pd.DataFrame({'unidade':lista_unidade, 'momento':lista_plt,
                               'unidade_count':lista_unidade_count, 'plt_count':lista_plt_count})

scrap_unidades['momento'] = scrap_unidades['momento'].apply(lambda x: x.replace('  ',' '))

scrap_unidades['momento'] = scrap_unidades['momento'].apply(lambda x: unidecode.unidecode(x.replace('  ',' ').upper().strip()))
scrap_unidades['unidade'] = scrap_unidades['unidade'].apply(lambda x: unidecode.unidecode(x.replace('  ',' ').upper().strip()))
print('OK')

OK


## Base

In [10]:
#CARREGANDO OS DADOS PARA SUBIR NO PEGA PLANTÃO
df = pd.read_excel('df_gabs.xlsx') #  # df_continuar
df = df.sort_values(['nome', 'data']).reset_index(drop=True)

#df['setor'][(df['setor'] == 'FDS - C.CIRURGICO ROTINA') & (df['unidade'] == 'SANTA ISABEL')] = 'FDS - MANHA - C.CIRURGICO'

print(len(df))
df.sample(3)

677


,chat,dia,nome,mensagem,controle,data,unidade,extra,tipo,ENTRADA,SAIDA,PERIODO,periodo,setor,sem_ou_fds,unidade2
666,[24/09/2024 17:16:31] Vivian Cristine dos Anjo...,24/09/2024,VIVIAN CRISTINE DOS ANJOS E SILVA,24/09 VILLA VIVIAN SAIDA 17:10,NaN,24/09/2024,VILLA,NaN,SAIDA,13:00,17:30,tarde,3,04 - SEMANA - CENTRO CIRURGICO,semana,VILLA
419,[23/09/2024 13:15:43] Maria Teresa Souza Gomes...,23/09/2024,MARIA TERESA SOUZA GOMES,23/09 OSASCO - M TERESA SAÍDA 13:10 ( PÓS ...,NaN,23/09/2024,OSASCO,NaN,SAIDA,13:00,13:30,tarde,3,04 - SEMANA - CENTRO CIRURGICO,semana,OSASCO
491,[24/09/2024 07:17:24] Pedro Henrique Lucas de ...,24/09/2024,PEDRO HENRIQUE LUCAS DE MORAES,24/09 ALPHA - MORAES ENTRADA 7:00,NaN,24/09/2024,ALPHA,NaN,ENTRADA,07:00,13:00,manha,2,04 - SEMANA - CENTRO CIRURGICO,semana,ALPHA


In [11]:
#TESTANDO COMPATIBILIDADE DOS DADOS CAPTURADOS E DADOS CARREGADOS
for X in range(0, len(df)):
    if len(scrap_unidades.loc[(scrap_unidades.unidade == df['unidade'][X]) & 
                                 (scrap_unidades.momento == df['setor'][X])]) < 1:
        print(X)
        print(df['unidade'][X], '-', df['setor'][X])
        #print('index', ind)
        #print(scrap_unidades['unidade'][ind], '-', scrap_unidades['momento'][ind])
        #print('codigos:', unidade, setorzin)

print("="*20, "TUDO OK", "="*20)

==================== TUDO OK ====================


# Funções Selenium

In [12]:
def subir_pg():
    browser.execute_script("window.scrollTo(0, 0);")

In [13]:
def reset_selecao():
    #xpath_click('/html')#; sleep(0.3)
    #xpath_click('/html')

    posicao_antiga = pyautogui.position() 
    pyautogui.click(x=200, y=158); sleep(0.3)
    pyautogui.click(x=200, y=158)
    pyautogui.moveTo(posicao_antiga) 


def fechar_caixa_de_selecao():
    # Localiza um elemento que é seguro para clicar (por exemplo, o corpo da página)
    elemento_seguro = browser.find_element(By.TAG_NAME, 'body')
    # Cria uma ação para clicar no elemento seguro
    ActionChains(browser).move_to_element(elemento_seguro).click().perform()


def reset_selecao1():
    reset_selecao2()
    #fechar_caixa_de_selecao(); sleep(0.3)
    #fechar_caixa_de_selecao()

    posicao_antiga = pyautogui.position() 
    pyautogui.click(x=200, y=158); sleep(0.3)
    pyautogui.click(x=200, y=158)
    pyautogui.moveTo(posicao_antiga)


def reset_selecao2():
    posicao_antiga = pyautogui.position() 
    pyautogui.click(x=200, y=158); sleep(0.3)
    pyautogui.click(x=200, y=158)
    pyautogui.moveTo(posicao_antiga) 
reset_selecao2()

In [14]:
#selecionando setor
def setor(unidade, setor):
    #print('base:', unidade, '-', setor)
    xpath_click('//*[@id="s2id_ddlServiceSector"]/a/span[1]')
    ind = scrap_unidades.loc[(scrap_unidades.unidade == unidade) & 
                             (scrap_unidades.momento == setor)].index.values[0]
    unidade_sel = scrap_unidades['unidade_count'][ind]
    setor_sel = scrap_unidades['plt_count'][ind]
    xpath_click('//*[@id="select2-drop"]/ul/li['+str(unidade_sel)+']/ul/li['+str(setor_sel)+']/div')   
setor("SANTA ISABEL", '04 - SEMANA - CENTRO CIRURGICO')

#inserir e selecionar profissional
def nome_func(nome_profissional):
    #listar todos
    xpath_click('//*[@id="service-ckb-list-all-owner"]')
    
    xpath_click('//*[@id="service-ckb-list-all-owner"]')
    xpath_click('//*[@id="service-ckb-list-all-owner"]')
    sleep(0.3)
    xpath_click('//*[@id="s2id_ddlProfessionalA"]/a/span[1]')
    xpath_send_keys('//*[@id="select2-drop"]/div/input', nome_profissional)
    #Dando enter
    xpath_send_keys('//*[@id="select2-drop"]/div/input', Keys.RETURN)
nome_func('OTTO MITTERMAYER')

#inserir horario e duração
def hora(inicio, duracao):
    #Inicio do plantão
    xpath_clear('//*[@id="txtServiceStartTime"]')
    xpath_send_keys('//*[@id="txtServiceStartTime"]', inicio)
    sleep(0.3)
    xpath_clear('//*[@id="txtServiceDuration"]')
    xpath_send_keys('//*[@id="txtServiceDuration"]', duracao)
hora('13:30', '05:00')

In [15]:
#inserir data
def data_func(data_selecionada):
    data_selecionada = str(data_selecionada)
    i_chato2 = str(5) #4 e 5
    i_chato = str(3) #2 e 3

    xpath_clear('/html/body/div['+i_chato2+']/div/div[2]/div[2]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div['+i_chato+']/input[1]')
    #xpath_clear('/html/body/div[4]/div/div[2]/div[2]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div[3]/input[2]')
    sleep(0.3)
    
    xpath_send_keys('/html/body/div['+i_chato2+']/div/div[2]/div[2]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div['+i_chato+']/input[1]', data_selecionada)
    #xpath_send_keys('/html/body/div[4]/div/div[2]/div[2]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div[3]/input[2]', data_selecionada)
    reset_selecao2()
data_func("12/03/2024")

In [16]:
def tipo_registro_func(i):
    xpath_click('//*[@id="s2id_add-service-service-types"]/a/span[1]')
    sleep(0.5)

    xpaths = [
        '/html/body/div[12]/div/input',
        '/html/body/div[11]/div/input',
        '/html/body/div[10]/div/input',
        '/html/body/div[9]/div/input',
        '/html/body/div[8]/div/input',
        '/html/body/div[7]/div/input',
        '/html/body/div[6]/div/input',
    ]

    for xpath in xpaths:
        try:
            xpath_send_keys(xpath, i)
            sleep(1)
            xpath_para_clicar = xpath[:xpath.find(']')+1] + '/ul/li/div'
            xpath_click(xpath_para_clicar)
            return  # Sai da função se encontrar um xpath que funcione
        except:
            continue  # Tenta o próximo xpath se ocorrer uma exceção
    
    print('erro')  # Imprime 'erro' se nenhum xpath funcionar

tipo_registro_func('VOLUNTÁRIO')

In [17]:
#inserindo pontuação
def pontuacao(pontos): # NÃO SE ALTERA MAIS
    #xpath_clear('//*[@id="txtServiceValue"]')
    #xpath_send_keys('//*[@id="txtServiceValue"]', pontos) #Pontos
    pass

pontuacao("666")

#selecionar tipo de plantão e ajustando tempo
def tipo_do_registro(duracao, sem_ou_fds, tipo_registro):
    if tipo_registro == 'extra':
        print('EXTRA')
        sleep(0.5)
        reset_selecao2()
        sleep(1)
        tipo_registro_func('VOLUNTÁRIO') #escrevendo
        if int(duracao[:2]) >= 5 and sem_ou_fds == "semana":
            pontuacao(200)
            pontuacao(200)
            print('duração >= 5 => 200 pontos')
        else:
            pass
    elif tipo_registro == 'DESLOCADO':
        pass
        #print('DESLOCADO')
        #sleep(1)
        #reset_selecao2()
        
        #xpath_click('//*[@id="s2id_add-service-service-types"]/a/span[1]')
        #xpath_send_keys('/html/body/div[11]/div/input', 'VOLUNTÁRIO') #escrevendo
        #xpath_click('/html/body/div[11]/ul/li/div') #clicando no primeiro
        #if int(duracao[:2]) >= 5 and sem_ou_fds == "semana":
        #    pontuacao(100)
        #    pontuacao(100)
        #    print('duração >5 -> 100 pontos')
        #else:
        #    pass
        #sleep(1)
    else:
        if int(duracao[:2]) >= 5 and sem_ou_fds == "semana":
            pontuacao(100)
            pontuacao(100)
            print('duração >5 -> 100 pontos')
        else:
            pass
        sleep(1)

#tipo_do_registro("05:00", "semana", "extra")
#tipo_do_registro(duracao, df['sem_ou_fds'][h], df['extra'][h])

In [18]:
def apagando_registro():
    chatao = str(5) #4 ou 5
    #Apagando
    browser.find_element("xpath", '/html/body/div['+chatao+']/div/div[1]/div[2]/button[1]').click()
    #Clicando em continuar
    sleep(0.5)
    browser.find_element("xpath", '/html/body/div['+chatao+']/div/div[1]/div[3]/button[2]').click()
#apagando_registro()

In [19]:
def insistindo_no_xpath(xpathzinho):
    xpath_click(xpathzinho)
    try:
        esperar_elemento_desaparecer(xpathzinho, 10)
    except:
        xpath_click(xpathzinho)

#########################################################

#salvando plantão
def salvar_plantao():
    xpathzinho = '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]'
    xpath_click(xpathzinho)
#salvar_plantao()

#fechando plantão
def fechar():
    xpathzinho = '//*[@id="modal-add-service"]/div/div[1]/button'
    xpath_click(xpathzinho)
fechar()

#atualizando plantão
xpath_atualizar = '/html/body/div[5]/div/div[1]/div[2]/button[5]'
#xpath_atualizar = '/html/body/div[4]/div/div[1]/div[2]/button[5]'
def atualizando_plantao():
    xpath_click(xpath_atualizar)
#atualizando_plantao()

# Função Tabela

In [20]:
######### ######### FUNÇÃO DE CAPTURA ######### ######### 

#pacotes para manipular data
from datetime import date
from datetime import time
from datetime import datetime
from datetime import timedelta

#capturando registros
unidades = [
            'ADMINISTRACAO','ALPHA','ANALIA',
            'CENTRAL DO TATUAPE',
            'HOSPITAIS EXTERNOS','HOSPITAL PARA TESTE',
            'ITAIM',
            'JABAQUARA',
            'MATERNIDADE','MORUMBI',
            'OSASCO',
            'SANTA ISABEL',
            'VILLA','VNS'
            ]

def funcao():
    lista_tabela = []
    for i in range(1, 6): #passando por cada semana[1 a 5]
    ##print('semana:', i, "=="*20)
        for t in range(1, 8): #passando por cada dia[1 a 7] dentro da semana
            reg = xpath_element('/html/body/form/main/div[2]/div[2]/table/tbody/tr['+str(i)+']/td['+str(t)+']').text
            content = reg.splitlines() #deixando as palavras em um item dentro de uma lista
            content = [unidecode.unidecode(i.strip().upper()) for i in content] #tratando cada palavra

            dia = content[0] #dia do registro
            del(content[0]) #deletando o dia, ja que foi coletado
            del(content[-1]) #deletando residuo do wepscraping

            if len(content) != 0: #ignorando dias em branco (sem registro) ¹
                #print('dia:', t)

                ##print('dia:', dia)
                #print(content)

                count_hora = 0 #count para capturar a hora de entrada e saida, count=1(entrada) | count=2(saida)
                lista_control = []
                for y in content: #passando um for sobre cada item da lista
                    if y in unidades: # capturando a unidade e dando inicio a novo registro novo
                        #print('================unidade:', y)
                        unidade = y
                    elif y[2] == ':': #reconhecendo se o item é hora (HH*:*mm)
                        count_hora += 1
                        if count_hora == 1: #capturando entrada
                            #print('hora entrada:', y)
                            lista_control.append(y)
                        elif count_hora == 2: #capturando saida
                            #print('hora saida:', y)
                            lista_control.append(y)
                            ##print('lista teste', lista_control)
                            lista_tabela.append(lista_control)
                            lista_control = []
                            count_hora = 0
                    else: #capturando dia, unidade, e periodo
                        #print('.')
                        #print('periodo:', y)
                        lista_control.append(dia)
                        lista_control.append(unidade)
                        lista_control.append(y)
                        lista_control.append(i)
                        lista_control.append(t) #dia semanal

                ##print('='*20)    

            else:
                continue #ignorando dias em branco (sem registro) ²
    
    tabela = pd.DataFrame(lista_tabela, columns = ['Dia','Unidade','Periodo','Semana','Dia Semana','Entrada','Saida'])
    
    #limpeza da tabela
    #########3
    tabela['Dia'] = tabela['Dia'].apply(lambda x: int(x))

    #recolhendo os dias do mes passado que ainda aparecem na primeira semana
    dias_mes_passado = list(tabela[(tabela['Dia'] > 20) & (tabela['Semana'] == 1)].index) 
    tabela.drop(dias_mes_passado, axis=0, inplace=True) #excluindo
    tabela.reset_index(drop=True, inplace=True) #resetando index

    #Inserindo a ordem dos registros
    tabela['ordem'] = ''

    for i in tabela['Dia'].unique():
        for t in range(0, len(tabela[tabela['Dia'] == i].index)):
            tabela['ordem'][tabela[tabela['Dia'] == i].index[t]] = t+1

    #Inserindo coluna de data        
    tabela['data'] = tabela['Dia'].apply(lambda x: str(x))

    for i in range(len(tabela)):
        if len(tabela['data'][i]) == 1:
            tabela['data'][i] = "0" + tabela['data'][i]

        tabela['data'][i] = tabela['data'][i] + "/" + mes + "/2024" 

    tabela = tabela.query("Unidade != 'ADMINISTRACAO'")
    tabela = tabela[~((tabela['Semana'] >= 5) & (tabela['Dia'] < 10))]
    tabela.reset_index(drop=True, inplace=True) #resetando index
    
    tabela['PERIODO'] = np.nan
    for i in range(0, len(tabela)):
        if int(tabela['Entrada'][i][:2]) <= 6:
            tabela['PERIODO'][i] = '6h'
        elif int(tabela['Entrada'][i][:2]) >= 7 and int(tabela['Entrada'][i][:2]) <= 12:
            tabela['PERIODO'][i] = 'manha'
        elif int(tabela['Entrada'][i][:2]) >= 13 and int(tabela['Entrada'][i][:2]) <= 18:
            tabela['PERIODO'][i] = 'tarde'
        else:
            tabela['PERIODO'][i] = 'noturno saida'

    tabela['periodo'] = np.nan
    tabela['periodo'][tabela['PERIODO'] == '6h'] = 1
    tabela['periodo'][tabela['PERIODO'] == 'manha'] = 2
    tabela['periodo'][tabela['PERIODO'] == 'tarde'] = 3
    tabela['periodo'][tabela['PERIODO'] == 'noturno saida'] = 4
    tabela['periodo'][tabela['PERIODO'] == 'noturno entrada'] = 5

    #################################################
    #print(tabela['Periodo'].unique())
    tabela['setor_inicio'] = np.nan
    for i in range(len(tabela)):
        corte = tabela['Periodo'][i].find('-')
        texto = tabela['Periodo'][i][:corte].strip()
        tabela['setor_inicio'][i] = texto
    #print(tabela['setor_inicio'].unique())
    #################################################
    ################################################################################################
    tabela['cor'] = np.nan
    def verificar_cor(dia_sel_filter):
        try:
            endereco = 'tr['+str(dia_sel_filter['Semana'][0])+']/td['+str(dia_sel_filter['Dia Semana'][0])+']/ul['+str(dia_sel_filter['ordem'][0])+']/li'
            xpath = '/html/body/form/main/div[2]/div[2]/table/tbody/'+endereco+'/div[2]/span'
            #print( str(dia_sel['Dia'][0]) )
            cor = xpath_element(xpath).get_attribute('style') #1#
            if cor == 'background-color: rgb(228, 11, 26);':
                return('vermelho')

        except:
            endereco = 'tr['+str(dia_sel_filter['Semana'][0])+']/td['+str(dia_sel_filter['Dia Semana'][0])+']/ul/li['+str(dia_sel_filter['ordem'][0])+']'
            xpath = '/html/body/form/main/div[2]/div[2]/table/tbody/'+endereco+'/div[2]/span'
            #print( str(dia_sel['Dia'][0]) , '----')
            try:
                cor = xpath_element(xpath).get_attribute('style') #2#
                if cor == 'background-color: rgb(228, 11, 26);':
                    return('vermelho')
            except:
                return('prob')
        return(cor)

    for i in range(len(tabela)):
        dia_sel_filter = tabela.loc[i:i+1].reset_index()
        tabela['cor'][i] = verificar_cor(dia_sel_filter)
    ################################################################################################
    
    return tabela
    
tabela = funcao()
tabela.head()

,Dia,Unidade,Periodo,Semana,Dia Semana,Entrada,Saida,ordem,data,PERIODO,periodo,setor_inicio,cor


# CAPTURANDO DADOS

## Nomes

In [21]:
def diferenca_horas(horario_inicial, horario_final):
    formato = "%H:%M"
    horario_inicial = datetime.strptime(horario_inicial, formato)
    horario_final = datetime.strptime(horario_final, formato)
    diferenca = horario_final - horario_inicial
    if diferenca.days < 0:
        diferenca = horario_inicial - horario_final
        return diferenca.total_seconds() / 60 * (-1)
    else:
        return diferenca.total_seconds() / 60

In [22]:
# Selecionar nome
def xpath_estranho(i):
    xpaths = [
        '/html/body/div[12]/div/input',
        '/html/body/div[11]/div/input',
        '/html/body/div[10]/div/input',
        '/html/body/div[9]/div/input',
        '/html/body/div[8]/div/input',
        '/html/body/div[7]/div/input',
        '/html/body/div[6]/div/input',
    ]
    
    for xpath in xpaths:
        try:
            xpath_send_keys(xpath, i)
            return  # Sai da função se encontrar um xpath que funcione
        except:
            continue  # Tenta o próximo xpath se ocorrer uma exceção
    
    print('erro')  # Imprime 'erro' se nenhum xpath funcionar
#xpath_estranho(i)

In [23]:
xpath_click('//*[@id="s2id_ddlProfessionals"]/a') #barra de nome
sleep(2)

html = browser.find_element("xpath", '//*[@id="select2-drop"]/ul').get_attribute('outerHTML') #
soup_nome = BeautifulSoup(html, 'html.parser')
#soup_nome
NOMES = []
for i in soup_nome.find_all(class_ ="select2-results-dept-0 select2-result select2-result-selectable"):
    #print(i.get_text())
    NOMES.append(i.get_text())
#del(NOMES[0])

reset_selecao()
display(NOMES)
print(len(NOMES))

['ALESSANDRE LUIZ BRAGA JUNIOR',
 'ALEXANDRE AMARAL LUCAS',
 'ALEXANDRE BERNARDINO',
 'ALEXSANDER TAKASHI HAMADA',
 'ALINE FRANCHINI GOULART LIMA',
 'AMANDA KIYOKO SOUZA KOSSAKA',
 'ANA CLARA MOURÃO BARRETO',
 'ANA GABRIELA GEWEHR VALE SALMEN',
 'ANA JUNG DOCKO',
 'ANDRÉ AUGUSTO BELLINI',
 'ANDRÉ DA COSTA AGUIAR VENTURA',
 'ANDRÉ LUIS XAVIER FRANCO',
 'ANDRÉ MORTARI PLÁ GIL',
 'ANDRÉIA LOUREIRO MORI',
 'ANNA BEATRIZ ARANHA TREVISAN',
 'ANTONIO ALVARO DIAS LIMA DA CUNHA NETO',
 'ARTHUR OLIVEIRA FACCHINI',
 'ARTHUR RODRIGUES TORRELIO',
 'AUGUSTO DE CASTRO SANTOS QUEIROZ',
 'BEATRIZ ANTIPOU DOS SANTOS',
 'BIANCA PRISCILA BUNIOTTI',
 'BRIAN SOSSAI PACHECO',
 'BRUCE YE MAN CHOW',
 'BRUNA DAIBERT DANESIN',
 'BRUNO FERRAZ DE CAMPOS BARONE',
 'BRUNO STORNIOLI TAGUCHI LEMOS',
 'CAIO TULIO DUARTE',
 'CAMILA CHIARANTANO HARRISON',
 'CAMILA RESENDE DE PAULA',
 'CARLOS AUGUSTO TREVISAN',
 'CARLOS EDUARDO CORRÊA GRILO',
 'CARLOS EDUARDO GONDIM OLIVEIRA',
 'CAROLINE FIRMINO LEMES DA SILVA',
 'CLAUDIO

241


In [24]:
len(df['nome'].unique())

182

In [25]:
def calculando_duracao(entrada, saida):
    # Calculando a diferença de tempo
    diferenca_tempo = datetime.strptime(saida, '%H:%M') - datetime.strptime(entrada, '%H:%M')
    
    # Convertendo a diferença para segundos
    diferenca_tempo_s = diferenca_tempo.seconds
    
    # Calculando horas e minutos
    horas = diferenca_tempo_s // 3600
    minutos = (diferenca_tempo_s % 3600) // 60
    
    # Formatando o tempo no formato HH:MM
    tempo = f'{horas:02}:{minutos:02}'
    
    return tempo

print(calculando_duracao('07:00', '12:00'))  # 05:00
print(calculando_duracao('08:00', '18:00'))  # 10:00
print(calculando_duracao('19:00', '06:00'))


05:00
10:00
11:00


# Capturando todos registros ja imputados

In [26]:
count_control = 0

tabelona = pd.DataFrame(columns=['Nome'] + list(funcao().columns))
tabelona

,Nome,Dia,Unidade,Periodo,Semana,Dia Semana,Entrada,Saida,ordem,data,PERIODO,periodo,setor_inicio,cor


In [27]:
count_control += 1

In [30]:
#for NOME in NOMES[count_control:]:
for NOME in df['nome'].unique()[count_control:]:

    if count_control > 0:
        if NOMES[count_control] != NOMES[count_control-1] or count_nome == 0:
            print(NOMES[count_control]) 
            #print('\n', '='*20, NOMES[count_control], '='*20, '\n')
            print(count_control, '|', len(NOMES)-count_control)

    xpath_click('//*[@id="s2id_ddlProfessionals"]/a/span[1]'); sleep(1)
    xpath_estranho(NOME)
    xpath_click('//*[@id="select2-drop"]/ul/li/div')

    WebDriverWait(browser, 30).until_not(visibility_of(xpath_element('//*[@id="loader"]')))
    
    tabela = funcao()
    tabela['Nome'] = NOME
    tabelona = pd.concat([tabela, tabelona])
    count_control += 1

tabelona['Nome'] = tabelona['Nome'].apply(lambda nome: unidecode.unidecode(nome).upper().strip().replace("  ", " "))
tabelona_backup = tabelona.copy
print(tabelona.shape)
tabelona.sample(5)

DIOGO HENRIQUE PEREIRA MATTOS
43 | 198
EDUARDO ARBACHE BEZERRA
44 | 197
EDUARDO DALA ROSA SOARES
45 | 196
ELAINE IMAEDA DE MOURA
46 | 195
ELISETE BEATRIZ VIANA
47 | 194
EMERSON DOMINGOS DA COSTA
48 | 193
ERICK OLIVEIRA LIMA GOMES
49 | 192
ESTANRLEY BARCELOS PINTO
50 | 191
ETTORI AUGUSTO ZACHARIAS DE ALMEIDA
51 | 190
EVERTON SIDNEY DA CONCEIÇÃO CARVALHO
52 | 189
FABIANA BARBOSA BASTOS
53 | 188
FABIANA SOARES GAMA
54 | 187
FÁBIO ANTÔNIO DE SOUZA OLIVEIRA
55 | 186
FÁBIO YUJI SUGUITA
56 | 185
FELIPE COELHO DE SOUZA
57 | 184
FELIPE DE OLIVEIRA NUNES
58 | 183
FELIPE RECH BORGES
59 | 182
FERNANDA TELES RIBEIRO
60 | 181
FERNANDO AKIRA KOGA
61 | 180
FERNANDO NARDY BELLICIERI
62 | 179
FERNANDO RIBEIRO SÁ
63 | 178
FILIPE LUGON MOULIN LIMA
64 | 177
FILIPE MAEDA HASHI ABDALLA MARTINS
65 | 176
FLÁVIO HIDEKI OSHIKA
66 | 175
FLÁVIO MAIA CASTILHO
67 | 174
FLÁVIO ROSA FIGUEIREDO SOUZA
68 | 173
FRANCISCO ANTONIO ALVES BONATTO
69 | 172
FRANCISCO JOSÉ LUCENA BEZERRA
70 | 171
GABRIEL DE AGUIAR SANTANA
71 | 

,Dia,Unidade,Periodo,Semana,Dia Semana,Entrada,Saida,ordem,data,PERIODO,periodo,setor_inicio,cor,Nome
21,17,OSASCO,04 - SEMANA - CENTRO CIRURGICO,4,2,07:00,19:00,2,17/09/2024,manha,2.0,04,"background-color: rgb(19, 34, 235);",ISAQUE CAMILO DA SILVA
14,11,SANTA ISABEL,04 - SEMANA - CENTRO CIRURGICO,3,3,07:00,19:00,1,11/09/2024,manha,2.0,04,"background-color: rgb(2, 184, 252);",KAIE FERNANDES
5,12,ITAIM,04 - SEMANA - CENTRO CIRURGICO,3,4,07:00,11:30,1,12/09/2024,manha,2.0,04,"background-color: rgb(120, 36, 134);",NATAN SOARES SILVEIRA
6,6,OSASCO,04 - SEMANA - CENTRO CIRURGICO,2,5,07:00,19:00,2,06/09/2024,manha,2.0,04,"background-color: rgb(175, 249, 79);",ARTHUR RODRIGUES TORRELIO
17,16,SANTA ISABEL,04 - SEMANA - CENTRO CIRURGICO,4,1,07:00,19:00,1,16/09/2024,manha,2.0,04,"background-color: rgb(2, 184, 252);",PAULA LOPES AGAPITO DE FREITAS


In [29]:
count_control += 1

# =======================================================================

# Verificações

# oque fazer com registros novos

In [31]:
#################################################
print(df['setor'].unique())
df['setor_inicio'] = np.nan
for i in range(len(df)):
    corte = df['setor'][i].find('-')
    texto = df['setor'][i][:corte].strip()
    df['setor_inicio'][i] = texto

print(df['setor_inicio'].unique())
#################################################

['04 - SEMANA - CENTRO CIRURGICO' '07 - NOTURNA - SAIDA APOS 19HS'
 '13 - FDS - CENTRO CIRURGICO' '11 - FDS - 06 HS'
 '01 - NOTURNA - ENTRADA ANTES DAS 07HS' '14 - FDS - SAIDA'
 '09 - NOTURNA - PLANTAO' '20 - FDS - PLANTAO']
['04' '07' '13' '11' '01' '14' '09' '20']


In [32]:
#################################
df['oque_fazer_com_o_registro'] = np.nan

for i in range(len(df)):

    nome = df['nome'][i]
    data = df['data'][i]
    periodo = df['PERIODO'][i]
    
    setor_inicio = df['setor_inicio'][i]
    filtro = tabelona['setor_inicio'] == setor_inicio

    if setor_inicio == '04':
        filtro1 = tabelona['setor_inicio'] == '02'
        filtro2 = tabelona['setor_inicio'] == '04.3'
        filtro3 = tabelona['setor_inicio'] == '06'
        # Combina o filtro original com os novos filtros
        filtro = filtro | filtro1 | filtro2 | filtro3

    tabela = tabelona[(tabelona['Nome'] == nome) & 
                      (tabelona['data'] == data) & 
                      ((tabelona['PERIODO'] == periodo) | filtro)]
    
                        #(tabelona['PERIODO'] == periodo)]
                    
    if len(tabela) == 0:
        df['oque_fazer_com_o_registro'][i] = 'criar plantao'

    elif len(tabela) == 1: # Verificando se os horarios batem
        if df['ENTRADA'][i] != tabela['Entrada'][tabela.index[0]] or df['SAIDA'][i] != tabela['Saida'][tabela.index[0]]:
            df['oque_fazer_com_o_registro'][i] = 'alterar horario'
        else:
            df['oque_fazer_com_o_registro'][i] = 'nada' # OK

    elif len(tabela) >= 2:
        df['oque_fazer_com_o_registro'][i] = 'apagar duplicata'
    else:

        print("WTF ?")
        break

count_freq(df, 'oque_fazer_com_o_registro')

,Freq,Perc
oque_fazer_com_o_registro,,
alterar horario,402,59.379616
criar plantao,186,27.474151
nada,86,12.703102
apagar duplicata,3,0.443131


In [33]:
df_filter = df[(df['oque_fazer_com_o_registro'] == 'alterar horario') | (df['oque_fazer_com_o_registro'] == 'nada')]
os.remove("df_gabs.xlsx")
salvando_nome_arquivo('df_gabs.xlsx', df_filter)

df_filter = df[df['oque_fazer_com_o_registro'] == 'criar plantao']
salvando_nome_arquivo('df_gabs2.xlsx', df_filter)

df_filter = df[df['oque_fazer_com_o_registro'] == 'apagar duplicata']
salvando_nome_arquivo('df_giov.xlsx', df_filter)

####################################################################
df = df[(df['oque_fazer_com_o_registro'] == 'alterar horario') | (df['oque_fazer_com_o_registro'] == 'nada')].reset_index(drop=True)

df_gabs.xlsx # arquivo criado #, len = 488
df_gabs2.xlsx # arquivo criado #, len = 186
df_giov.xlsx # arquivo atualizado #, len = 160  +=  3 => 163


In [34]:
df_backup = df.copy()

# Diminuindo Tempos & Deslocado

# Execução

In [35]:
count = 0
print(df_backup.shape)

(488, 18)


In [36]:
############################-CODIGO-############################
#Para cada profissional na lista
df = df_backup[count:]

for i in df['nome'].unique():
    print('='*20, i, '='*20)
    df_nome = df[df['nome'] == i]
    df_nome = df_nome.reset_index(drop=True)

    ###########################################
    # MUDANDO O PROFISSIONAL ESCOLHIDO
    subir_pg(); sleep(0.5)
    try:
        xpath_click('//*[@id="s2id_ddlProfessionals"]/a/span[1]'); sleep(1)
    except:
        fechar()
        sleep(1)
        df_backup['controle'][count-1] = "PROBS"; print(count-1, "-> PROBS")
        xpath_click('//*[@id="s2id_ddlProfessionals"]/a/span[1]'); sleep(1)
    xpath_estranho(i)
    xpath_click('//*[@id="select2-drop"]/ul/li/div')
    ###########################################
    
    # esperando carregamento do site
    WebDriverWait(browser, 30).until_not(visibility_of(xpath_element('//*[@id="loader"]')))
    sleep(2)
    # Capturando calendario de registros do individuo
    tabela = funcao(); sleep(0.3)
    
    # Para cada registro
    for t in range(len(df_nome)):
        ##########################################################################################################
        print(df_nome['mensagem'][t])

        '''
        # Analisandos os registros no **dia**
        dia_sel = tabela[tabela['data'] == df_nome['data'][t]] # & (tabela['Periodo'] == df_nome["setor"][t])]
        # Encurtando entrada - quando entrou atrasado
        if df_nome['PERIODO'][t] == 'manha' and df_nome['sem_ou_fds'][t] == 'fds':
            dia_sel = dia_sel[dia_sel['Entrada'] == '07:00']
        else:
            dia_sel = dia_sel[dia_sel['PERIODO'] == df_nome["PERIODO"][t]]
        '''

        ####################################################
        setor_inicio = df_nome['setor_inicio'][t]
        filtro = tabela['setor_inicio'] == setor_inicio

        if setor_inicio == '04':
            filtro1 = tabela['setor_inicio'] == '02'
            filtro2 = tabela['setor_inicio'] == '04.3'
            filtro3 = tabela['setor_inicio'] == '06'
            # Combina o filtro original com os novos filtros
            filtro = filtro | filtro1 | filtro2 | filtro3

        dia_sel = tabela[(tabela['data'] == df_nome['data'][t]) &
                        ((tabela['PERIODO'] == df_nome['PERIODO'][t]) | filtro)]
        ####################################################

        dia_sel.reset_index(drop=True, inplace=True)        
        ##########################################################################################################

        if len(dia_sel) == 0:
            df_backup['controle'][count] = "PROB"; print(count, "-> PROB")
        elif len(dia_sel) == 2:
            if df_backup['tipo'][count] == 'SAIDA':
                # abrindo/selecionando periodo conforme localização
                try:
                    xpath_localizacao = '//*[@id="shift-monthly"]/div[2]/table/tbody/tr['+str(dia_sel['Semana'][1])+']/td['+str(dia_sel['Dia Semana'][1])+']/ul['+str(dia_sel['ordem'][1])+']/li'
                    esperar_elemento_aparecer(xpath_localizacao, 5)
                    xpath_click(xpath_localizacao)
                except:
                    xpath_localizacao = '//*[@id="shift-monthly"]/div[2]/table/tbody/tr['+str(dia_sel['Semana'][1])+']/td['+str(dia_sel['Dia Semana'][1])+']/ul/li['+str(dia_sel['ordem'][1])+']'
                    esperar_elemento_aparecer(xpath_localizacao, 5)
                    xpath_click(xpath_localizacao)
                # Esperar componentes carregarem
                WebDriverWait(browser, 10).until(visibility_of(xpath_element('//*[@id="service-ckb-list-all-owner"]')))
                sleep(0.3)
                
                #apagando_registro(); sleep(0.3)
                tipo_registro_func('DISPENSA'); sleep(0.5); atualizando_plantao()
                
                df_backup['controle'][count] = "OK"; print(count, "-> OK")
                WebDriverWait(browser, 10).until(visibility_of(xpath_element('//*[@id="s2id_ddlProfessionals"]/a/span[1]')))
                sleep(1)
            else:
                df_backup['controle'][count] = "OK"; print(count, "-> OK")
        else:
            #abrindo/selecionando periodo conforme localização
            try:
                xpath_localizacao = '//*[@id="shift-monthly"]/div[2]/table/tbody/tr['+str(dia_sel['Semana'][0])+']/td['+str(dia_sel['Dia Semana'][0])+']/ul['+str(dia_sel['ordem'][0])+']/li'
                esperar_elemento_aparecer(xpath_localizacao, 5)
                xpath_click(xpath_localizacao)
            except:
                xpath_localizacao = '//*[@id="shift-monthly"]/div[2]/table/tbody/tr['+str(dia_sel['Semana'][0])+']/td['+str(dia_sel['Dia Semana'][0])+']/ul/li['+str(dia_sel['ordem'][0])+']'
                esperar_elemento_aparecer(xpath_localizacao, 5)
                xpath_click(xpath_localizacao)
            #Esperar componentes carregarem
            WebDriverWait(browser, 10).until(visibility_of(xpath_element('//*[@id="service-ckb-list-all-owner"]')))
            sleep(1)

            if df_nome['tipo'][t] == 'ENTRADA':
                duracao = calculando_duracao(df_nome['ENTRADA'][t], dia_sel['Saida'][0])
                hora(df_nome['ENTRADA'][t], duracao)
                #print(1)
            elif df_nome['tipo'][t] == 'SAIDA':
                duracao = calculando_duracao(dia_sel['Entrada'][0], df_nome['SAIDA'][t])
                hora(dia_sel['Entrada'][0], duracao)
                #print(2)
            elif df_nome['tipo'][t] == 'ENTRADA_SAIDA':
                duracao = calculando_duracao(df_nome['ENTRADA'][t], df_nome['SAIDA'][t])
                hora(df_nome['ENTRADA'][t], duracao)
                print('int:', df_nome['ENTRADA'][t], ' -> ', df_nome['SAIDA'][t])
                #print(3)
            
            #if dia_sel['Unidade'][0] != df_nome['unidade'][t]:
                #setor(df_nome['unidade'][t], df_nome['setor'][t])
            #else:
                #pass
            
            tipo_do_registro(duracao, df_nome['sem_ou_fds'][t], df_nome['extra'][t])
            sleep(0.3)
            reset_selecao2(); sleep(0.3); reset_selecao2()
            #salvar plantão
            sleep(0.3)
            atualizando_plantao(); atualizando_plantao(); sleep(2)

            try:
                if WebDriverWait(browser, 10).until_not(EC.visibility_of_element_located((By.XPATH, xpath_atualizar))) == False:
                    pass
            except:
                atualizando_plantao()
                try: 
                    if WebDriverWait(browser, 10).until_not(EC.visibility_of_element_located((By.XPATH, xpath_atualizar))) == False:
                        pass
                except:
                    break
            chatao = 5 #6 5 4
            if xpath_element('/html/body/div['+str(chatao)+']/div/div[2]/div[1]').text != "":
                fechar(); fechar()
                sleep(1)
                df_backup['controle'][count] = "PROBS"; print(count, "-> PROBS")
            else:
                df_backup['controle'][count] = "OK"; print(count, "-> OK")
        
        count += 1 
        
        if count % 25 == 0:
            print('='*80, '\n', df_backup['controle'].value_counts(), '\n'); df_backup.to_excel("df_gabs.xlsx", index=False)
        else:
            if t != len(df_nome)-1:
                print('='*20)

print("="*50, 'TERMINOU', "="*50)
print('Total  ', len(df_backup))
print('='*80, df_backup['controle'].value_counts(), '='*80); df_backup.to_excel("df_gabs.xlsx", index=False)

==================== ALEXSANDER TAKASHI HAMADA ====================
21/09 ANALIA  -  ALEX ENTRADA 07:00
0 -> OK
21/09 ANALIA  -  ALEX SAÍDA 13:40
1 -> OK
22/09 MORUMBI  -  ALEX ENTRADA 05:30
2 -> OK
23/09 ANALIA ALEX ENTRADA 07:00
duração >5 -> 100 pontos
3 -> OK
24/09 ANALIA  -  ALEX ENTRADA 07:00
duração >5 -> 100 pontos
4 -> OK
==================== ALINE FRANCHINI GOULART LIMA ====================
21/09 ANÁLIA ALINE SAÍDA 13:10
5 -> OK
24/09 VILLA ALINE ENTRADA 05:35
6 -> OK
24/09 VILLA ALINE SAÍDA 15:10
duração >5 -> 100 pontos
7 -> OK
==================== AMANDA KIYOKO SOUZA KOSSAKA ====================
20/09 MORUMBI  -  AMANDA ENTRADA 09:00
duração >5 -> 100 pontos
8 -> OK
20/09 MORUMBI  -  AMANDA SAÍDA 23:40
duração >5 -> 100 pontos
9 -> OK
24/09 MORUMBI  -  AMANDA ENTRADA 09:00
duração >5 -> 100 pontos
10 -> OK
24/09 MORUMBI  -  AMANDA SAÍDA 18:10
duração >5 -> 100 pontos
11 -> OK
==================== ANA JUNG DOCKO ====================
21/09 VILLA ANA ENTRADA 05:00
12 -> OK
21

In [ ]:
df_backup['controle'][count] = "OK"; print(count, "-> OK")
#df_backup['controle'][count] = "PROBS"; print(count, "-> PROBS")
count += 1

# XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

In [ ]:
count = 0

In [ ]:
############################-CODIGO-############################
#Para cada profissional na lista
df = df_backup[count:]

for i in df['nome'].unique():
    print('='*20, i, '='*20)
    df_nome = df[df['nome'] == i]
    df_nome = df_nome.reset_index(drop=True)

    ###########################################
    # MUDANDO O PROFISSIONAL ESCOLHIDO
    subir_pg(); sleep(0.5)
    xpath_click('//*[@id="s2id_ddlProfessionals"]/a/span[1]'); sleep(1)
    xpath_estranho(i)
    xpath_click('//*[@id="select2-drop"]/ul/li/div')
    ###########################################
    
    # esperando carregamento do site
    WebDriverWait(browser, 30).until_not(visibility_of(xpath_element('//*[@id="loader"]')))
    sleep(2)
    # Capturando calendario de registros do individuo
    tabela = funcao(); sleep(0.3)
    
    # Para cada registro
    for t in range(len(df_nome)):
        ##########################################################################################################
        print(df_nome['mensagem'][t])

        '''
        # Analisandos os registros no **dia**
        dia_sel = tabela[tabela['data'] == df_nome['data'][t]] # & (tabela['Periodo'] == df_nome["setor"][t])]
        # Encurtando entrada - quando entrou atrasado
        if df_nome['PERIODO'][t] == 'manha' and df_nome['sem_ou_fds'][t] == 'fds':
            dia_sel = dia_sel[dia_sel['Entrada'] == '07:00']
        else:
            dia_sel = dia_sel[dia_sel['PERIODO'] == df_nome["PERIODO"][t]]
        '''

        ####################################################
        dia_sel = tabela[(tabela['data'] == df_nome['data'][t]) & (tabela['Entrada'] == df_nome['ENTRADA'][t])]
        ####################################################

        dia_sel.reset_index(drop=True, inplace=True)        
        ##########################################################################################################

        if len(dia_sel) == 0:
            df_backup['controle'][count] = "PROB"; print(count, "-> PROB")
        elif len(dia_sel) == 2:
            # abrindo/selecionando periodo conforme localização
            try:
                xpath_localizacao = '//*[@id="shift-monthly"]/div[2]/table/tbody/tr['+str(dia_sel['Semana'][0])+']/td['+str(dia_sel['Dia Semana'][0])+']/ul['+str(dia_sel['ordem'][0])+']/li'
                esperar_elemento_aparecer(xpath_localizacao, 5)
                xpath_click(xpath_localizacao)
            except:
                xpath_localizacao = '//*[@id="shift-monthly"]/div[2]/table/tbody/tr['+str(dia_sel['Semana'][0])+']/td['+str(dia_sel['Dia Semana'][0])+']/ul/li['+str(dia_sel['ordem'][0])+']'
                esperar_elemento_aparecer(xpath_localizacao, 5)
                xpath_click(xpath_localizacao)
            # Esperar componentes carregarem
            WebDriverWait(browser, 10).until(visibility_of(xpath_element('//*[@id="service-ckb-list-all-owner"]')))
            sleep(0.3)
            
            #apagando_registro(); sleep(0.3)
            tipo_registro_func('DISPENSADO'); sleep(0.5); atualizando_plantao()
            
            df_backup['controle'][count] = "OK"; print(count, "-> OK")
            WebDriverWait(browser, 10).until(visibility_of(xpath_element('//*[@id="s2id_ddlProfessionals"]/a/span[1]')))
            sleep(1)
        else:
            #abrindo/selecionando periodo conforme localização
            try:
                xpath_localizacao = '//*[@id="shift-monthly"]/div[2]/table/tbody/tr['+str(dia_sel['Semana'][0])+']/td['+str(dia_sel['Dia Semana'][0])+']/ul['+str(dia_sel['ordem'][0])+']/li'
                esperar_elemento_aparecer(xpath_localizacao, 5)
                xpath_click(xpath_localizacao)
            except:
                xpath_localizacao = '//*[@id="shift-monthly"]/div[2]/table/tbody/tr['+str(dia_sel['Semana'][0])+']/td['+str(dia_sel['Dia Semana'][0])+']/ul/li['+str(dia_sel['ordem'][0])+']'
                esperar_elemento_aparecer(xpath_localizacao, 5)
                xpath_click(xpath_localizacao)
            #Esperar componentes carregarem
            WebDriverWait(browser, 10).until(visibility_of(xpath_element('//*[@id="service-ckb-list-all-owner"]')))
            sleep(1)
            hora(df_nome['ENTRADA'][t], df_nome['DURAÇÃO'][t])
            if dia_sel['Unidade'][0] != df_nome['unidade'][t]:
                setor(df_nome['unidade'][t], df_nome['setor'][t])
            else:
                pass
            
            tipo_do_registro(df_nome['DURAÇÃO'][t], df_nome['sem_ou_fds'][t], df_nome['extra'][t])
            sleep(0.3)
            reset_selecao2()
            #salvar plantão
            sleep(0.3)
            atualizando_plantao(); atualizando_plantao(); sleep(2)
            df_backup['controle'][count] = "OK"; print(count, "-> OK")

            xpath_atualizar = '/html/body/div[5]/div/div[1]/div[2]/button[5]'
            try:
                if WebDriverWait(browser, 10).until_not(EC.visibility_of_element_located((By.XPATH, xpath_atualizar))) == False:
                    pass
            except:
                atualizando_plantao()
                try: 
                    if WebDriverWait(browser, 10).until_not(EC.visibility_of_element_located((By.XPATH, xpath_atualizar))) == False:
                        pass
                except:
                    break

            sleep(1)
        
        count += 1 
        
        if count % 25 == 0:
            print('='*80, '\n', df_backup['controle'].value_counts(), '\n'); df_backup.to_excel("df_gabs.xlsx", index=False)
        else:
            if t != len(df_nome)-1:
                print('='*20)
            
print("="*50, 'TERMINOU', "="*50)
print('Total  ', len(df_backup))
print('='*80, df_backup['controle'].value_counts(), '='*80); df_backup.to_excel("df_gabs.xlsx", index=False)

In [ ]:
df_backup.loc[int(count), 'controle'] = "OK"
count += 1
print(df_backup['nome'][count])
print(count)

## Salvando

In [37]:
df_corrigidos = df_backup[df_backup['controle'] == "OK"]
print("probs ok:", len(df_corrigidos))

df_prob = df_backup[df_backup['controle'] != "OK"]
print("probs:", len(df_prob))

###############################################################

os.remove("df_gabs.xlsx")
salvando_nome_arquivo('Feitos.xlsx', df_corrigidos)
print('.')
salvando_nome_arquivo('df_gabs2.xlsx', df_prob)

probs ok: 435
probs: 53
Feitos.xlsx # arquivo atualizado #, len = 3888  +=  435 => 4323
.
df_gabs2.xlsx # arquivo atualizado #, len = 186  +=  53 => 239


# ============================

# Plantões Normais

In [38]:
df = pd.read_excel('df_gabs2.xlsx')
df = df.sort_values(['nome', 'data']).reset_index(drop=True)
df.shape

(239, 18)

In [39]:
print(len(df))
df = df.drop(df[df['data'] == '30/09/2024'].index, axis=0).reset_index(drop=True)
df = df.drop(df[df['data'] == '31/09/2024'].index, axis=0).reset_index(drop=True)
print(len(df))

239
239


In [40]:
def subtrair_um_dia(data_str):
    data = pd.to_datetime(data_str, format='%d/%m/%Y')  # Converte para datetime
    data -= pd.Timedelta(days=1)  # Subtrai um dia
    return data.strftime('%d/%m/%Y')  # Converte de volta para string

lista_linhas_except = []
for linha in df[pd.isnull(df['ENTRADA'])].index:
    try:
        df['data'][linha] = subtrair_um_dia(df['data'][linha])
        df['ENTRADA'][linha] = '19:00'
    except:
        lista_linhas_except.append(linha)

display(df.loc[lista_linhas_except])

,chat,dia,nome,mensagem,controle,data,unidade,extra,tipo,ENTRADA,SAIDA,PERIODO,periodo,setor,sem_ou_fds,unidade2,setor_inicio,oque_fazer_com_o_registro


# Execução

In [41]:
H=0 

In [43]:
count = 0
for h in range(H, len(df)):
    #####################################################################################################################
    if h > 0:
        if df['nome'][h] != df['nome'][h-1] or count == 0:
            print('='*20, df['nome'][h], '='*20)
            count += 1
    #####################################################################################################################
    #try:
    print('[',h,'] |','dia:', df['data'][h],'| mensagem:', df['mensagem'][h])
    
    #click criar novo plantão
    criar_plantao()
    
    #SETOR - UNIDADE
    #esta disponivel o botão de setor ?
    esperar_elemento_aparecer('//*[@id="s2id_ddlServiceSector"]/a/span[1]', 10)
    #Selecionando Setor
    setor(df['unidade'][h], df['setor'][h])
    
    #PROFISSIONAL
    #esta disponivel o botão de profissional
    esperar_elemento_aparecer('//*[@id="service-ckb-list-all-owner"]', 10)
    nome_func(df['nome'][h])
    sleep(1)

    #DATA DO PLANTÃO
    data_func(df['data'][h])
    
    #INICIO DO PLANTÃO -- DURAÇÃO DO PLANTÃO
    duracao = calculando_duracao(df['ENTRADA'][h], df['SAIDA'][h])
    hora(df['ENTRADA'][h], duracao)
    sleep(1)
        
    #TIPO
    tipo_do_registro(duracao, df['sem_ou_fds'][h], df['extra'][h])
    
    #SALVAR PLANTÃO
    sleep(1)
    salvar_plantao()
    salvar_plantao()
    salvar_plantao()
    sleep(4)

    #VERIFICANDO SE HOUVE ERRO - HÁ PLANTÃO NO MESMO HORARIO
    chatao = 5 #6 5 4
    if xpath_element('/html/body/div['+str(chatao)+']/div/div[2]/div[1]').text != "":
        fechar()
        df.loc[int(h), 'controle'] = "PROBS"; print("PROB")
    else:
        df.loc[int(h), 'controle'] = "NORM"; print("NORM")
    #####################################################################################################################
    '''except Exception as e:
        print(f"Erro ao processar o arquivo: {e}")
        if existencia('//*[@id="modal-add-service"]/div/div[1]/button'):
            fechar()
            print('*'*50)
            df.loc[int(h), 'controle'] = "ERRO_REPETIR"; print("ERRO_REPETIR")
            print('*'*50)
        else:
            print('-VERIFICAR-'*10)
            break'''
            
    #####################################################################################################################
    # praticamente um # finally
    WebDriverWait(browser, 10).until_not(visibility_of(xpath_element('//*[@id="modal-add-service"]/div/div[1]/button')))
    H += 1
    #####################################################################################################################
    if H % 25 == 0:
        print('='*80, '\n', df['controle'].value_counts(), '\n', '='*80); df.to_excel("df_gabs2.xlsx", index=False)
    if df['nome'][h] == df['nome'][h+1]:
        print('='*10)

print("="*20, 'TERMINOU', "="*20)
print('Total  ', len(df))
print('='*80, '\n', df['controle'].value_counts(), '\n', '='*80); df.to_excel("df_gabs2.xlsx", index=False)

==================== TERMINOU ====================
Total   239
 controle
NORM     162
PROBS     77
Name: count, dtype: int64 


In [ ]:
df.loc[int(h), 'controle'] = "NORM"; print("NORM")
#df.loc[int(h), 'controle'] = "PROBS"; print("PROB")
H += 1

## Salvando

In [44]:
print(df['controle'].value_counts())
print(df['controle'].value_counts(normalize=True)*100)

count_freq(df, 'controle')

controle
NORM     162
PROBS     77
Name: count, dtype: int64
controle
NORM     67.782427
PROBS    32.217573
Name: proportion, dtype: float64


,Freq,Perc
controle,,
NORM,162,67.782427
PROBS,77,32.217573


In [45]:
df_corrigidos = df[df['controle'] == "NORM"]
print("probs ok:", len(df_corrigidos))

df_prob = df[df['controle'] != "NORM"]
print("probs:", len(df_prob))

os.remove("df_gabs2.xlsx")
###############################################################

salvando_nome_arquivo('Feitos.xlsx', df_corrigidos)
print('.')
salvando_nome_arquivo('df_giov2.xlsx', df_prob)

probs ok: 162
probs: 77
Feitos.xlsx # arquivo atualizado #, len = 4323  +=  162 => 4485
.
df_giov2.xlsx # arquivo atualizado #, len = 66  +=  77 => 143


# =======================

# Dispensados

In [46]:
df_backup = pd.read_excel('df_dispensados.xlsx')
df_backup = df_backup.drop_duplicates(subset=['chat'])
df_backup = df_backup.sort_values(['nome','data']).reset_index(drop=True)

print(df_backup.shape)
df_backup.head()

(4, 8)


,chat,dia,nome,mensagem,controle,controle2,data,hora
0,[23/09/2024 10:56:13] Claudio Mueller Kakuda: ...,23/09/2024,CLAUDIO MUELLER KAKUDA,21/09 MATERNIDADE - KAKUDA DISPENSADO,DISP_SAUDE,OK,21/09/2024,NaN
1,[21/09/2024 10:20:28] Francisco Antonio Alves ...,21/09/2024,FRANCISCO ANTONIO ALVES BONATTO,21/09 SANTA - FRANCISCO DISPENSADO,DISP_SAUDE,OK,21/09/2024,NaN
2,[21/09/2024 07:21:17] Nathália Fanucci Oliveir...,21/09/2024,NATHALIA FANUCCI OLIVEIRA DI JORGE,21/09 OSASCO NATHÁLIA DISPENSADO,DISP_SAUDE,OK,21/09/2024,NaN
3,[21/09/2024 10:40:34] Thassio Cunha de Santana...,21/09/2024,THASSIO CUNHA DE SANTANA,21/09 VILLA THASSIO DISPENSADO,DISP_SAUDE,OK,21/09/2024,NaN


In [47]:
count = 0
df_backup['controle'] = np.nan

# Execução

def apagando_registro():
    chatao = str(5) #4 ou 5
    #Apagando
    browser.find_element("xpath", '/html/body/div['+chatao+']/div/div[1]/div[2]/button[1]').click()
    #Clicando em continuar
    sleep(0.5)
    browser.find_element("xpath", '/html/body/div['+chatao+']/div/div[1]/div[3]/button[2]').click()
apagando_registro()

In [48]:
len(df_backup)

4

In [51]:
#Para cada profissional na lista
df = df_backup[count:]

for i in df['nome'].unique():
    print('='*20, i, '='*20)
    df_nome = df[df['nome'] == i]
    df_nome.reset_index(drop=True, inplace=True)
    
    xpath_click('//*[@id="s2id_ddlProfessionals"]/a/span[1]'); sleep(1)
    xpath_estranho(i)
    xpath_click('//*[@id="select2-drop"]/ul/li/div')

    #esperando carregamento do site
    WebDriverWait(browser, 30).until_not(visibility_of(xpath_element('//*[@id="loader"]')))

    tabela = funcao()    
    sleep(2)
    
    #Para cada registro na lista_nome
    for t in range(len(df_nome)):
        print(df_nome['mensagem'][t])
        dia_sel = tabela[tabela['data'] == df_nome['data'][t]]
        dia_sel.reset_index(drop=True, inplace=True)
        
        if len(dia_sel) == 0:
            print(count, "-> Ja apagado")
            print(df_nome['chat'][t] + " - " + df_nome['data'][t])
            df_backup['controle'][count] = "OK"
        else:
            for r in range(len(dia_sel)-1, -1, -1):
                print('PRINCIPAL')
                #abrindo/selecionando periodo conforme localização
                dia_ordem = r
                xpath_textao = '//*[@id="shift-monthly"]/div[2]/table/tbody/tr['+str(dia_sel['Semana'][0])+']/td['+str(dia_sel['Dia Semana'][0])+']/ul['+str(dia_sel['ordem'][dia_ordem])+']/li'
                xpath_click(xpath_textao)
                #Esperar componentes carregarem
                WebDriverWait(browser, 10).until(visibility_of(xpath_element('//*[@id="service-ckb-list-all-owner"]')))
                sleep(2)
                reset_selecao2()
                tipo_registro_func('DISPENSA')
                sleep(0.3)
                reset_selecao2()
                #salvar plantão
                sleep(0.3)
                atualizando_plantao(); atualizando_plantao(); sleep(2)
                try:
                    if WebDriverWait(browser, 10).until_not(EC.visibility_of_element_located((By.XPATH, xpath_atualizar))) == False:
                        pass
                except:
                    atualizando_plantao()
                    try: 
                        if WebDriverWait(browser, 10).until_not(EC.visibility_of_element_located((By.XPATH, xpath_atualizar))) == False:
                            pass
                    except:
                        break
            
            tabela = funcao() #atualizando tabela de registros
            sleep(2)
            print(count, '-> OK')
            print(df_nome['chat'][t] + " - " + df_nome['data'][t])
            df_backup['controle'][count] = "OK"  
            
        count += 1 
        if count % 25 == 0:
            print('='*80, '\n', df_backup['controle'].value_counts(), '\n', '='*80); df_backup.to_excel("df_dispensados.xlsx", index=False)
        else:
            if t != len(df_nome)-1:
                print('='*20)
    
print('='*25, 'TERMINOU', '='*25)
print('Total  ', len(df_backup))
print('='*80, df_backup['controle'].value_counts(), '='*80); df_backup.to_excel("df_dispensados.xlsx", index=False)

==================== THASSIO CUNHA DE SANTANA ====================
21/09 VILLA THASSIO DISPENSADO
PRINCIPAL
3 -> OK
[21/09/2024 10:40:34] Thassio Cunha de Santana: 21/09 Villa Thassio dispensado - 21/09/2024
========================= TERMINOU =========================
Total   4
================================================================================ controle
OK       3
PROBS    1
Name: count, dtype: int64 ================================================================================


In [50]:
df_backup['controle'][count] = "OK"
df_backup['controle'][count] = "PROBS"
count += 1

## Salvando

In [52]:
df_corrigidos = df_backup[df_backup['controle'] == "OK"]
print("probs ok:", len(df_corrigidos))

df_prob = df_backup[df_backup['controle'] != "OK"]
print("probs:", len(df_prob))

################################################################

os.remove("df_dispensados.xlsx")
salvando_nome_arquivo('Feitos.xlsx', df_corrigidos)
print('.')
salvando_nome_arquivo('df_giov.xlsx', df_prob)

probs ok: 3
probs: 1
Feitos.xlsx # arquivo atualizado #, len = 4485  +=  3 => 4488
.
df_giov.xlsx # arquivo atualizado #, len = 163  +=  1 => 164


# =============================

# AUTOMAÇÕES PARA RODAR POUCAS VEZES POR MÊS

## ACERTANDO UNIDADES DO DIA

In [ ]:
df_total = pd.read_excel('Feitos.xlsx')
df_total.head(2)

count_control = 0

In [ ]:
reset_selecao()
lista = []
count_nome = 0 

for NOME in NOMES[count_control:]:

    if count_control > 0:
        if NOMES[count_control] != NOMES[count_control-1] or count_nome == 0:
            print('\n', '='*20, NOMES[count_control], '='*20, '\n')
            count += 1
            
    xpath_click('//*[@id="s2id_ddlProfessionals"]/a/span[1]'); sleep(1)
    xpath_estranho(NOME)
    xpath_click('//*[@id="select2-drop"]/ul/li/div')

    #esperando carregamento do site
    WebDriverWait(browser, 30).until_not(visibility_of(browser.find_element("xpath", '//*[@id="loader"]')))
    
    tabela = funcao()
    
    count = 0
    for dia in tabela['Dia'].unique():
        tabela = funcao()
        tabela = tabela[tabela['Unidade'] != 'HOSPITAIS EXTERNOS'].reset_index(drop=True)
        tabela = tabela[tabela['Dia Semana'] < 6].reset_index(drop=True)
        
        dia_sel = tabela[tabela['Dia'] == dia].reset_index(drop=True)
        if len(dia_sel['Unidade'].unique()) > 1:
            print('alterando unidades, dia:', dia)

            while len(dia_sel['Unidade'].unique()) > 1:
                
                condicao = df_total['nome'] == unidecode.unidecode(NOME.upper())
                condicao1 = df_total['data'] == dia_sel['data'].values[0]
                unidade_do_dia = df_total['unidade'][condicao & condicao1]
                if len( unidade_do_dia ) == 0:
                    print('VERIFICAR'*5)
                    break
                unidade_do_dia = unidade_do_dia.values[0]
                dia_sel2 = dia_sel[dia_sel['Unidade'] != unidade_do_dia]
                
                print(dia_sel2['Unidade'].values[0], '=>', unidade_do_dia)
                #abrindo/selecionando periodo conforme localização
                xpath_localizacao = '//*[@id="shift-monthly"]/div[2]/table/tbody/tr['+str(dia_sel2['Semana'].values[0])+']/td['+str(dia_sel2['Dia Semana'].values[0])+']/ul['+str(dia_sel2['ordem'].values[0])+']/li'
                xpath_click(xpath_localizacao)
                
                #Esperar componentes carregarem
                WebDriverWait(browser, 10).until(visibility_of(xpath_element('//*[@id="service-ckb-list-all-owner"]')))
                sleep(0.5)
                
                #Selecionando Setor
                periodozin = dia_sel2['Periodo'].values[0]
                #periodozin = periodozin.replace('COBERTURA EXTRA','C.CIRURGICO') 
                #periodozin = periodozin.replace('AV.PRE ANESTESICA','C.CIRURGICO')
                try:
                    setor(unidade_do_dia, periodozin)

                    if diferenca_horas(dia_sel2['Entrada'].values[0], dia_sel2['Saida'].values[0]) >= 5*60:
                        pontuacao(100)
                        pontuacao(100)
                        
                    #salvar plantão
                    sleep(1)
                    atualizando_plantao()
                    atualizando_plantao()
                    WebDriverWait(browser, 10).until(visibility_of(xpath_element('//*[@id="s2id_ddlProfessionals"]/a/span[1]')))
                    sleep(5)
                except:
                    fechar()

                tabela = funcao()
                tabela = tabela[tabela['Dia Semana'] < 6].reset_index(drop=True)
                dia_sel = tabela[tabela['Dia'] == dia].reset_index(drop=True)
                
    count_control += 1

## Verificando registros imputados, vendo se esta tudo ok

## Registros pareados - entrada_saida

In [ ]:
df = pd.read_excel('Feitos.xlsx')
df['controle2'][pd.isnull(df['controle2'])] = 'PROB'
df[df['oque_fazer_com_o_registro'] == 'dispensa']['controle2'] = 'OK'
print(df['controle2'].value_counts())

In [ ]:
# encontrando eles
######################################################################################################
# dias com uma entrada e uma saida
count_erro = 0
lista_count_errop = []
count_erro1 = 0
lista_count_erro1p = []
count_erro2 = 0
lista_count_erro2p = []
count_ok = 0
lista_count_ok = []

for medico in df['nome'].unique():
    df_medico = df[df['nome'] == medico]
    for data in df_medico['data'].unique():
        df_filter = df_medico[df_medico['data'] == data]

        if len(df_filter) > 2:
            count_erro2 += 1
            for ind in df_filter.index: lista_count_erro2p.append(ind)
        elif len(df_filter) == 2:
            teste1 = len(df_filter[df_filter['tipo'] == 'ENTRADA']) == 1
            teste2 = len(df_filter[df_filter['tipo'] == 'SAIDA']) == 1
            if teste1 and teste2:
                count_ok += 1
                for ind in df_filter.index: lista_count_ok.append(ind)
            else:
                count_erro += 1
                for ind in df_filter.index: lista_count_errop.append(ind)
        else:
            count_erro1 += 1
            for ind in df_filter.index: lista_count_erro1p.append(ind)

print('tudo certo, lista_count_ok:', count_ok, len(lista_count_ok), lista_count_ok) ######*************
print('erro ==2, lista_count_errop:', count_erro, len(lista_count_errop), lista_count_errop)
print('erro <2, lista_count_erro1p:', count_erro1, len(lista_count_erro1p), lista_count_erro1p)
print('erro >2, lista_count_erro2p:', count_erro2, len(lista_count_erro2p), lista_count_erro2p)
print('=-='*35)
######################################################################################################
# dias com entrada e saida entre [7h e 19h]
count_analisavel = 0
lista_count_analisavel = []
count_n_analisavel = 0
lista_count_n_analisavel = []
dff = df.iloc[lista_count_ok] ######*************

for medico in dff['nome'].unique():
    df_medico = dff[dff['nome'] == medico]
    for data in df_medico['data'].unique():
        df_filter = df_medico[df_medico['data'] == data]

        id_entrada = df_filter[df_filter['tipo'] == 'ENTRADA'].index[0]
        id_saida = df_filter[df_filter['tipo'] == 'SAIDA'].index[0]

        condicao1 =  float(df_filter['ENTRADA'][id_entrada].replace(':','.')) >= 7.00
        condicao2 =  float(df_filter['SAIDA'][id_saida].replace(':','.')) <= 19.00
        if condicao1 and condicao2:
            count_analisavel += 1
            for ind in df_filter.index: lista_count_analisavel.append(ind)
        else:
            count_n_analisavel += 1
            for ind in df_filter.index: lista_count_n_analisavel.append(ind)

print('lista_count_analisavel:', count_analisavel, len(lista_count_analisavel), lista_count_analisavel) ######*************
print('lista_count_n_analisavel:', count_n_analisavel, len(lista_count_n_analisavel), lista_count_n_analisavel)
print('=-='*35)
######################################################################################################
# destes registros, quantos estão presentes realmente corretamente no pega ?
count_ok = 0
count_erro1 = 0
count_erro = 0
lista_count_erro = []

dff = df.iloc[lista_count_analisavel] ######*************

for medico in dff['nome'].unique():
    df_medico = dff[dff['nome'] == medico]
    for data in df_medico['data'].unique():
        df_filter = df_medico[df_medico['data'] == data]

        id_entrada = df_filter[df_filter['tipo'] == 'ENTRADA'].index[0]
        id_saida = df_filter[df_filter['tipo'] == 'SAIDA'].index[0]

        tabela = tabelona[(tabelona['Nome'] == medico) & 
                          (tabelona['data'] == data) & 
                          (tabelona['Entrada'] == df['ENTRADA'][id_entrada]) &
                          (tabelona['Saida'] == df['SAIDA'][id_saida])]
        
        if len(tabela) == 1:
            count_ok += 1
            for id in df_filter.index: df['controle2'][id] = 'OK'
        elif len(tabela) > 1:
            count_erro1 += 1 
        else:
            count_erro += 1
            for id in df_filter.index: lista_count_erro.append(id)

print(count_ok)
print(count_erro1)
print(count_erro, 'lista_count_erro:', lista_count_erro) ######*************
print('=-='*35)
######################################################################################################
# 
dff = df.iloc[lista_count_erro] ######*************
count_ok = 0
lista_count_ok = []
count_erro = 0
lista_count_erro = []
count_erro2 = 0
lista_count_erro2 = []

for medico in dff['nome'].unique():
    df_medico = dff[dff['nome'] == medico]
    for data in df_medico['data'].unique():
        df_filter = df_medico[df_medico['data'] == data]

        id_entrada = df_filter[df_filter['tipo'] == 'ENTRADA'].index[0]
        id_saida = df_filter[df_filter['tipo'] == 'SAIDA'].index[0]

        tabela = tabelona[(tabelona['Nome'] == medico) & 
                          (tabelona['data'] == data)]
        
        if len(tabela) == 2:
            # entrada e saida em dois lançamentos
            teste1 = len(tabela[tabela['Entrada'] == df['ENTRADA'][id_entrada]]) == 1
            teste2 = len(tabela[tabela['Saida'] == df['SAIDA'][id_saida]]) == 1
            if teste1 and teste2:
                count_ok += 1
                for id in df_filter.index:  df['controle2'][id] = 'OK'
            else:
                count_erro += 1
                for id in df_filter.index: lista_count_erro.append(id)
        else:
            count_erro2 += 1
            for id in df_filter.index: lista_count_erro2.append(id)

print(count_ok)
print(count_erro, 'lista_count_erro:', lista_count_erro) ######*************
print(count_erro2, 'lista_count_erro2:', lista_count_erro2) ######*************
print('=-='*35)
######################################################################################################
# colocando tudo em um data_frame para atualizar os dados pega
df_af = pd.DataFrame(columns = df.columns)
dff = df.iloc[lista_count_erro + lista_count_erro2] ######*************

for medico in dff['nome'].unique():
    df_medico = dff[dff['nome'] == medico]
    for data in df_medico['data'].unique():
        df_filter = df_medico[df_medico['data'] == data]

        id_entrada = df_filter[df_filter['tipo'] == 'ENTRADA'].index[0]
        id_saida = df_filter[df_filter['tipo'] == 'SAIDA'].index[0]

        df_af.loc[len(df_af)] = list(df.iloc[id_entrada])
        df_af['ENTRADA'][len(df_af)-1] = df['ENTRADA'][id_entrada]
        df_af['SAIDA'][len(df_af)-1] = df['SAIDA'][id_saida]


df_af['tipo'] = 'ENTRADA_SAIDA'
df_af.shape
df_backup = df_af
######################################################################################################


In [ ]:
# corrigir via programa - celula acima
df['controle2'].iloc[lista_count_erro + lista_count_erro2] = 'OK'

# corrigir manualmente
#df[['chat','mensagem','nome','data','ENTRADA','SAIDA','tipo']].iloc[lista_count_errop]
df['controle2'].iloc[lista_count_errop] = 'OK'

# enviar pro otto
df['controle2'].iloc[lista_count_erro2p] = 'OTTO'

In [ ]:
df['controle2'].value_counts()

In [ ]:
# VISUALIZANDO ARQUIVO
df[['chat','mensagem','nome','data','ENTRADA','SAIDA','tipo']].iloc[lista_count_erro2p]

# encontrando restantes e definindo erros reais

In [ ]:
count_n_encontrado = 0
count_encontrado = 0
count_2_encontrado = 0
count_erro_estranho = 0

for i in df.iloc[lista_count_erro1p + lista_count_n_analisavel].index:

    if df['controle2'][i] == 'OK':
        continue

    nome = df['nome'][i]
    data = df['data'][i]
    periodo = df['PERIODO'][i]
    
    setor_inicio = df['setor_inicio'][i]
    filtro = tabelona['setor_inicio'] == setor_inicio

    if setor_inicio == '04':
        filtro1 = tabelona['setor_inicio'] == '02'
        filtro2 = tabelona['setor_inicio'] == '04.3'
        filtro3 = tabelona['setor_inicio'] == '06'
        # Combina o filtro original com os novos filtros
        filtro = filtro | filtro1 | filtro2 | filtro3

    tabela = tabelona[(tabelona['Nome'] == nome) & 
                      (tabelona['data'] == data) & 
                      ((tabelona['PERIODO'] == periodo) | filtro)]

    if len(tabela) == 0:
        count_n_encontrado += 1

    elif len(tabela) == 1: # Verificando se os horarios batem
        if df['tipo'][i] == 'ENTRADA':
           if df['ENTRADA'][i] == tabela['Entrada'][tabela.index[0]]:
               df['controle2'][i] = 'OK'
               count_encontrado += 1
        elif df['tipo'][i] == 'SAIDA':
           if df['SAIDA'][i] == tabela['Saida'][tabela.index[0]]:
               df['controle2'][i] = 'OK'
               count_encontrado += 1
        else:
            count_erro_estranho += 1
    elif len(tabela) >= 2:
        count_2_encontrado += 1
    else:
        count_erro_estranho += 1

print(count_n_encontrado)
print(count_encontrado)
print(count_2_encontrado)

In [ ]:
df['controle2'].value_counts()

In [ ]:
df[df['controle2'] == 'PROB'][['chat','mensagem','nome','data','ENTRADA','SAIDA','tipo']].head(30)

# =============================

# Encerrando Automação

In [53]:
browser.quit()

# ==============================

# Residuos Finais

In [54]:
df_feito = pd.read_excel("Feitos.xlsx")
print(len(df_feito))

df_prob0 = pd.read_excel("df_giov.xlsx")
print(len(df_prob0))
os.remove("df_giov.xlsx")

df_prob01 = pd.read_excel("df_giov2.xlsx")
print(len(df_prob01))
os.remove("df_giov2.xlsx")

df_prob = pd.concat([df_prob0, df_prob01])
df_prob = df_prob.drop_duplicates(subset=['chat'])
salvando_nome_arquivo('df_prob.xlsx', df_prob)

print(len(df_prob))

4488
164
143
df_prob.xlsx # arquivo criado #, len = 307
307


In [55]:
df_prob = pd.read_excel('df_prob.xlsx')
print(len(df_prob))

307


In [56]:
df_prob_bk = df_prob.copy()
#df_prob = df_prob.iloc[-22:]

In [58]:
total = len(df_feito) + len(df_prob)

percent = len(df_feito) / total
percent = round(percent*100, 2)
percent = str(percent) + "%"
percent = "("+percent+")"
print("Registros App:", len(df_feito), percent)

percent = len(df_prob) / total
percent = round(percent*100, 2)
percent = str(percent) + "%"
percent = "("+percent+")"
print("Registros Giovanelli", len(df_prob), percent)
print("#"*30, "\n")

for nome in sorted(list(df_prob['nome'].unique())):
    print(nome, "="*10)
    selecao = df_prob.loc[df_prob['nome']==nome]
    selecao.reset_index(drop=True, inplace=True)
    
    for i in range(len(selecao)):
        print(selecao["mensagem"][i])
    
    print("\n")

Registros App: 4488 (93.6%)
Registros Giovanelli 307 (6.4%)
############################## 

ALESSANDRE LUIZ BRAGA JUNIOR ==========
19/09 ANALIA  -  ALESSANDRE ENTRADA 06:00 ‎<MENSAGEM EDITADA>


ALEXANDRE AMARAL LUCAS ==========
09/09 MORUMBI  -  ALEXANDRE AMARAL ENTRADA 06:00


ALEXANDRE BERNARDINO ==========
06/09  -  ALEXANDRE  -  ENTRADA 12:00  -  ANESTESIA EXTERNA
06/09  -  ALEXANDRE  -  SAÍDA 21:10  -  ANESTESIA EXTERNA
13/09 SANTA  -  ALEXANDRE  -  ENTRADA 19:00
23/09  -  VILLA  -  ALEXANDRE  -  ENTRADA 07:00
24/09  -  VILLA  -  ALEXANDRE  -  SAÍDA 01:45


ALEXSANDER TAKASHI HAMADA ==========
01/09 ANALIA  -  ALEX SAÍDA 19:00
22/09 MORUMBI  -  ALEX SAÍDA 13:40


ANA JUNG DOCKO ==========
17/09 ANALIA ANA ENTRADA 07:00
23/09 ANALIA ANA SAÍDA 11:10


ANDRE DA COSTA AGUIAR VENTURA ==========
21/09 MORUMBI VENTURA SAIDA 20:00


ANDREIA LOUREIRO MORI ==========
08/09 JABAQUARA ANDRÉIA SAÍDA 13:40  ( D2 ) 


ARTHUR OLIVEIRA FACCHINI ==========
21/09 VNS FACCHINI ENTRADA 07:00


ARTH

In [56]:
os.remove("df_prob.xlsx")
df_prob.to_excel("df_prob_enviado_3008.xlsx", index=False)

In [ ]:
df_feito = pd.read_excel("Feitos.xlsx")
print(len(df_feito))

df_prob = pd.read_excel("df_prob_enviado.xlsx")
print(len(df_prob))

total = len(df_feito) + len(df_prob)

percent = len(df_feito) / total
percent = round(percent*100, 2)
percent = str(percent) + "%"
percent = "("+percent+")"
print("Registros App:", len(df_feito), percent)

percent = len(df_prob) / total
percent = round(percent*100, 2)
percent = str(percent) + "%"
percent = "("+percent+")"
print("Registros Giovanelli", len(df_prob), percent)
print("#"*30, "\n")

In [ ]:
len(df_prob)

# =================================

# Teste

In [ ]:
df_prob = pd.read_excel('df_giov.xlsx')
for nome in sorted(list(df_prob['nome'].unique())):
    print(nome, "="*10)
    selecao = df_prob.loc[df_prob['nome']==nome]
    selecao.reset_index(drop=True, inplace=True)
    
    for i in range(len(selecao)):
        print(selecao["mensagem"][i])
    
    print("\n")

In [ ]:
len(df_prob)